# AUC: Current RedZone V2 model

Load the labeled dataset produced in notebook 01 and evaluate the current AutoGluon model AUC.


In [1]:
from __future__ import annotations

import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.metrics import roc_auc_score

# --- Config ---
DATA_PATH = "/Users/starsrain/jan2026_concord/jan2026_loonie_customize/retrain_data/features_with_fpdaa.parquet"
MODEL_PATH = "/Users/starsrain/jan2026_concord/jan2026_loonie_customize/src/model/autogluon_models_FPDAA_20250904_010918"
LABEL_COL = "FPDAA"


# --- Load data ---
df = pd.read_parquet(DATA_PATH)

# --- Load model ---
predictor = TabularPredictor.load(MODEL_PATH, require_py_version_match=False)
model_features = list(predictor.features())

# --- Align features ---
for col in model_features:
    if col not in df.columns:
        df[col] = 0
X = df[model_features]

# --- Predict probabilities ---
proba = predictor.predict_proba(X, model="CatBoost_r137_BAG_L1_FULL")
positive_class = getattr(predictor, "positive_class", None)
if positive_class in proba.columns:
    y_score = proba[positive_class]
else:
    y_score = proba.iloc[:, -1]

# --- Compute AUC ---
auc = roc_auc_score(df[LABEL_COL].astype(int), y_score)
print("rows:", len(df))
print("auc:", auc)


/Users/starsrain/jan2026_concord/jan2026_loonie_customize/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found 2 mismatches between original and current metadata:


rows: 5953
auc: 0.5972375560469769
